# Description
After running K fold to select those data which are predicted into a wrong category, we implemented mannual recorrection. This script is to apply those correction to the original files. 

In [1]:
import pandas as pd
import numpy as np

In [34]:
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
# model_list = ['CutDebt','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
df_others = pd.read_csv('../../MLModel/data/others/irrelevant_response_training_set.csv')
df_other_matrix = pd.read_csv('../../MLModel/data/others/strategy_mat.csv')
num2text = df_other_matrix.set_index('label').category.drop_duplicates().to_dict()

df_all_original = {}
for each in model_list:
    df_all_original[each] = pd.read_csv('../../MLModel/data/{}/mock_up_data1.csv'.format(each))

In [35]:
model_list = ['IDClassifier']
for model in model_list:
    df_correction = pd.read_csv('../../MLModel/data/{}/labelNeedsCorrections0719.csv'.format(model))
    other_label = max(df_all_original[model].label) + 1
    non_other_label = list(range(other_label))
    df_all_original[model] = df_all_original[model][df_all_original[model].label != other_label]
    print('{} -- {}'.format(model,len(df_all_original[model])))
    print('length of other: {}'.format(len(df_others)))
    append_other_text = []
    append_other_category = []

    append_function_text = {}
    append_function_category = {}

    original_remove_index = {}
    others_remove_index = []

    count_case1 = 0
    count_case2 = 0
    count_case3 = 0
    count_case4 = 0
    count_case5 = 0
    count_case6 = 0
    count_case7 = 0
    for index in range(len(df_correction)):
        ############ get data #####################
        cur_data = df_correction.iloc[index]
        data_from = cur_data['from']
        label = cur_data['label']
        original_text = cur_data['original_text']
        new_label = cur_data['new_label']
        original_label = cur_data['original_label']
        ########### apply to original files #######
        if model in ['CutDebt','Installment'] and new_label == 110:
            count_case4 += 1
            if label in non_other_label:
                ### insert into others
                append_other_text.append(original_text)
                append_other_category.append(num2text[new_label])
                ### remove from original
                rindex = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
                if original_remove_index.get(data_from) is None:
                    original_remove_index[data_from] = []
                original_remove_index[data_from].extend(list(rindex))
            else:
                ### update others
                rindex = df_others[df_others['文本']==original_text].index.values
                if len(rindex) == 0:
                    ### insert into others
                    append_other_text.append(original_text)
                    append_other_category.append(num2text[new_label])
                else:    
                    df_others.loc[rindex,'类别'] = num2text[110]
            continue
            
            
            
        # move from functional to other
        if label in non_other_label and new_label >= 100:
            count_case1 += 1
            ### insert into others
            append_other_text.append(original_text)
            append_other_category.append(num2text[new_label])

            ### remove from original
            rindex = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
            if original_remove_index.get(data_from) is None:
                original_remove_index[data_from] = []
            original_remove_index[data_from].extend(list(rindex))
            
            
            
        # delete non_other_label
        elif label in non_other_label and new_label == -1:
            ## remove from original
            count_case5 += 1
            rindex = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
            if original_remove_index.get(data_from) is None:
                original_remove_index[data_from] = []
            original_remove_index[data_from].extend(list(rindex))
            
        # delete other_label
        elif label == other_label and new_label == -1:
            if original_label != 109:
                count_case6 += 1
                rindex_other = df_others[(df_others['类别']==num2text[int(original_label)]) &(df_others['文本']==original_text)].index.values
                others_remove_index.extend(list(rindex_other))
        


        # update from functional 1 to functional 2
        elif label in non_other_label and new_label in non_other_label and label!= new_label:
            count_case2 += 1
            # update original data
            indexes = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
            df_all_original[data_from].loc[indexes,'label'] = new_label
            
            
            
            
        # update from other to functional
        elif label == other_label and new_label in non_other_label:
            count_case3+=1
            if append_function_text.get(model) is None:
                append_function_text[model]=[]
                append_function_category[model]=[]
            append_function_text[model].append(original_text)
            append_function_category[model].append(new_label)
            
            # remove from others
            if original_label != 109:
                rindex_other = df_others[(df_others['类别']==num2text[int(original_label)]) &(df_others['文本']==original_text)].index.values
                others_remove_index.extend(list(rindex_other))
            

    others_append = pd.DataFrame({'文本':append_other_text,'类别':append_other_category})
    function_append = pd.DataFrame({'split_text':append_function_text,'label':append_function_category})
    
    
    
    

    #### apply to self and others
    #### drop logic
    for each_key in original_remove_index:
        indexes = original_remove_index[each_key]
        df_all_original[each_key] = df_all_original[each_key].drop(index=indexes)
        print('{} -- {}'.format(each_key,len(df_all_original[each_key])))
    ####end drop
    #### drop others
    
    df_others = df_others.drop(index=others_remove_index)
    ### end drop others
    df_others = pd.concat([df_others,others_append],sort=True).reset_index(drop=True)
    print('length of other: {}'.format(len(df_others)))
    for each_key in append_function_text:
        function_append = pd.DataFrame({'split_text':append_function_text[each_key],
                                        'label':append_function_category[each_key]})

        df_all_original[each_key] = pd.concat([df_all_original[each_key],function_append],sort=True).reset_index(drop=True)
        print('{} -- {}'.format(each_key,len(df_all_original[each_key])))

    print('case 1 <function to others> has data : {}'.format(count_case1))
    print('case 2 <function to function> has data : {}'.format(count_case2))
    print('case 3 <others to function> has data : {}'.format(count_case3))
    print('case 4 has data : {}'.format(count_case4))
    print('case 5 <deleted from function> delted data: {}'.format(count_case5))
    print('case 6 <deleted from other>: {}'.format(count_case6))
    print('total corrected is: {}'.format(count_case1+count_case2+count_case3+count_case4))
    print('total file length is: {}'.format(len(df_correction)))


IDClassifier -- 2367
length of other: 5391
IDClassifier -- 2180
length of other: 5489
IDClassifier -- 2278
case 1 <function to others> has data : 121
case 2 <function to function> has data : 14
case 3 <others to function> has data : 98
case 4 has data : 0
case 5 <deleted from function> delted data: 19
case 6 <deleted from other>: 5
total corrected is: 233
total file length is: 748


In [48]:
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
for model in model_list:
    print(model)
    print(df_all_original[model].label.value_counts())
    print('===================')

CutDebt
1    2351
0    1817
Name: label, dtype: int64
IDClassifier
1    1261
0    1176
Name: label, dtype: int64
IfKnowDebtor
0    1413
1    1236
Name: label, dtype: int64
Installment
1    2422
0    1872
Name: label, dtype: int64
ConfirmLoan
0    718
1    551
Name: label, dtype: int64
WillingToPay
1    2677
0    1803
2    1084
Name: label, dtype: int64


In [29]:
len(df_correction[df_correction.new_label==-1])

38

In [46]:
# Save 
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
for model in model_list:
    df = df_all_original[model].copy()
    df['label'] = df['label'].astype('int')
    df.to_csv('../../MLModel/data/{}/mock_up_data1.csv'.format(model),index=False,encoding='utf8')
    print(model)
df_others.to_csv('../../MLModel/data/others/irrelevant_response_training_set.csv',index=False,encoding='utf8')

CutDebt
IDClassifier
IfKnowDebtor
Installment
ConfirmLoan
WillingToPay


# Single Model Evaluation

In [5]:
model = 'IDClassifier'
df_correction = pd.read_csv('../../MLModel/data/{}/labelNeedsCorrections0719.csv'.format(model))

In [7]:
df_correction[df_correction['new_label']==-1]

,from,label,original_label,original_text,new_label,Unnamed: 5
3,IDClassifier,1,NaN,不认识,-1,NaN
45,故意岔开话题,2,109.0,三天时间太短了，多给我三天肯定能处理,-1,NaN
56,IDClassifier,1,NaN,不认识,-1,NaN
57,IDClassifier,1,NaN,那天处理不了，在等今天把,-1,NaN
81,IDClassifier,1,NaN,2小时之内不行,-1,NaN
92,回问身份,2,107.0,我是欠钱，但是只欠公司的钱，不欠一个随便打电话来的陌生人的钱,-1,NaN
130,故意岔开话题,2,109.0,明天朋友就换我钱了,-1,NaN
131,故意岔开话题,2,109.0,能不能分期,-1,NaN
178,IDClassifier,0,NaN,是又怎么样，你们高利贷，我不还,-1,NaN
236,IDClassifier,0,NaN,这样吧，多给我四天，我肯定能还,-1,NaN


# Expriment - no other

In [11]:
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
# model_list = ['CutDebt','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
df_others = pd.read_csv('../../MLModel/data/others/irrelevant_response_training_set.csv')
df_other_matrix = pd.read_csv('../../MLModel/data/others/strategy_mat.csv')
num2text = df_other_matrix.set_index('label').category.drop_duplicates().to_dict()

df_all_original = {}
for each in model_list:
    df_all_original[each] = pd.read_csv('../../MLModel/data/{}/mock_up_data2.csv'.format(each))

In [14]:
for model in model_list:
    df_correction = pd.read_csv('../../MLModel/data/{}/labelNeedsCorrections.csv'.format(model))
    non_other_label = set(df_all_original[model].label.values) 
    print('{} -- {}'.format(model,len(df_all_original[model])))
    print('length of other: {}'.format(len(df_others)))
    append_other_text = []
    append_other_category = []

    append_function_text = {}
    append_function_category = {}

    original_remove_index = {}

    count_case1 = 0
    count_case2 = 0
    count_case3 = 0
    count_case4 = 0
    for index in range(len(df_correction)):
        ############ get data #####################
        cur_data = df_correction.iloc[index]
        data_from = cur_data['from']
        label = cur_data['label']
        original_text = cur_data['original_text']
        new_label = cur_data['new_label']
        ########### apply to original files #######
        if model in ['CutDebt','Installment'] and new_label == 110:
            count_case4 += 1
            if label in non_other_label:
                ### insert into others
                append_other_text.append(original_text)
                append_other_category.append(num2text[new_label])
                ### remove from original
                rindex = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
                if original_remove_index.get(data_from) is None:
                    original_remove_index[data_from] = []
                original_remove_index[data_from].extend(list(rindex))
            else:
                ### update others
                rindex = df_others[df_others['文本']==original_text].index.values
                if len(rindex) == 0:
                    ### insert into others
                    append_other_text.append(original_text)
                    append_other_category.append(num2text[new_label])
                else:    
                    df_others.loc[rindex,'类别'] = num2text[110]
            continue
        if label in non_other_label and new_label >= 100:
            count_case1 += 1
            ### insert into others
            append_other_text.append(original_text)
            append_other_category.append(num2text[new_label])

            ### remove from original
            rindex = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
            if original_remove_index.get(data_from) is None:
                original_remove_index[data_from] = []
            original_remove_index[data_from].extend(list(rindex))


        elif label in non_other_label and new_label in non_other_label and label!= new_label:
            count_case2 += 1
            # update original data
            indexes = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
            df_all_original[data_from].loc[indexes,'label'] = new_label

        

    others_append = pd.DataFrame({'文本':append_other_text,'类别':append_other_category})
    function_append = pd.DataFrame({'split_text':append_function_text,'label':append_function_category})

    #### apply to self and others
    #### drop logic
    for each_key in original_remove_index:
        indexes = original_remove_index[each_key]
        df_all_original[each_key] = df_all_original[each_key].drop(index=indexes)
        print('{} -- {}'.format(each_key,len(df_all_original[each_key])))
    ####end drop
    df_others = pd.concat([df_others,others_append],sort=True).reset_index(drop=True)
    print('length of other: {}'.format(len(df_others)))
    for each_key in append_function_text:
        function_append = pd.DataFrame({'split_text':append_function_text[each_key],
                                        'label':append_function_category[each_key]})

        df_all_original[each_key] = pd.concat([df_all_original[each_key],function_append],sort=True).reset_index(drop=True)
        print('{} -- {}'.format(each_key,len(df_all_original[each_key])))

    print('case 1 has data : {}'.format(count_case1))
    print('case 2 has data : {}'.format(count_case2))
    print('case 3 has data : {}'.format(count_case3))
    print('case 4 has data : {}'.format(count_case4))
    print('total corrected is: {}'.format(count_case1+count_case2+count_case3+count_case4))
    print('total file length is: {}'.format(len(df_correction)))


CutDebt -- 4166
length of other: 8096
CutDebt -- 4166
length of other: 8336
case 1 has data : 225
case 2 has data : 94
case 3 has data : 0
case 4 has data : 21
total corrected is: 340
total file length is: 1371
IDClassifier -- 2435
length of other: 8336
IDClassifier -- 2435
length of other: 8710
case 1 has data : 374
case 2 has data : 48
case 3 has data : 0
case 4 has data : 0
total corrected is: 422
total file length is: 1008
IfKnowDebtor -- 2649
length of other: 8710
IfKnowDebtor -- 2649
length of other: 8813
case 1 has data : 103
case 2 has data : 61
case 3 has data : 0
case 4 has data : 0
total corrected is: 164
total file length is: 605
Installment -- 4291
length of other: 8813
Installment -- 4291
length of other: 9051
case 1 has data : 233
case 2 has data : 262
case 3 has data : 0
case 4 has data : 7
total corrected is: 502
total file length is: 1655
ConfirmLoan -- 1269
length of other: 9051
ConfirmLoan -- 1269
length of other: 9069
case 1 has data : 18
case 2 has data : 29
case 

# Resotre

In [60]:
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']



df_need_correction_original = {}
df_need_correction_new = {}
for each in model_list:
    df_need_correction_original[each] = pd.read_csv('../../MLModel/data/{}/labelNeedsCorrections1.csv'.format(each))
    df_need_correction_new[each] = pd.read_csv('../../MLModel/data/{}/labelNeedsCorrections.csv'.format(each))
    df_all_original[each] = pd.read_csv('../../MLModel/data/{}/mock_up_data1.csv'.format(each))



In [67]:
index_remove = {}
for model in model_list:   
    
    df_tmp = df_need_correction_new[model].copy()
    for index in range(len(df_need_correction_original[model])):
        data = df_need_correction_original[model].iloc[index]
        data_from = data['from']
        label = data['label']
        original_text = data['original_text']
        filterd = df_tmp[(df_tmp['from'] == data_from) & (df_tmp.label == label) & (df_tmp.original_text == original_text) ]
        # if not in original
        if len(filterd) == 0:
            if data_from in model_list:
                #add key to index_remove
                df_ori = df_all_original[data_from].copy()
                if index_remove.get(data_from) is None:
                    index_remove[data_from] = []
                
                
                #check in original     
                filterd_ori = df_ori[ (df_ori.label == label) & (df_ori.split_text == original_text) ]
                if len(filterd_ori) > 0:
                    index_remove[data_from].extend(list(filterd_ori.index.values))
            
    print(model)
    print(len(index_remove[model]))
    print('=========================')
    


CutDebt
79
IDClassifier
72
IfKnowDebtor
112
Installment
263
ConfirmLoan
8
WillingToPay
160


In [68]:
#### drop logic
for each_key in index_remove:
    print('Before droping! {} -- {}'.format(each_key,len(df_all_original[each_key])))
    indexes = index_remove[each_key]
    df_all_original[each_key] = df_all_original[each_key].drop(index=indexes)
    print('After droping! {} -- {}'.format(each_key,len(df_all_original[each_key])))
####end drop

Before droping! CutDebt -- 4168
After droping! CutDebt -- 4111
Before droping! IDClassifier -- 2437
After droping! IDClassifier -- 2367
Before droping! IfKnowDebtor -- 2649
After droping! IfKnowDebtor -- 2547
Before droping! Installment -- 4294
After droping! Installment -- 4218
Before droping! ConfirmLoan -- 1269
After droping! ConfirmLoan -- 1261
Before droping! WillingToPay -- 5564
After droping! WillingToPay -- 5436


In [71]:
# Save 
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
for model in model_list:
    df = df_all_original[model].copy()
    df['label'] = df['label'].astype('int')
    df.to_csv('../../MLModel/data/{}/mock_up_data1.csv'.format(model),index=False,encoding='utf8')
    print(model)
    print(df.label.value_counts())
    print('===========')

CutDebt
1    2327
0    1784
Name: label, dtype: int64
IDClassifier
1    1234
0    1133
Name: label, dtype: int64
IfKnowDebtor
0    1363
1    1184
Name: label, dtype: int64
Installment
1    2379
0    1839
Name: label, dtype: int64
ConfirmLoan
0    713
1    548
Name: label, dtype: int64
WillingToPay
1    2620
0    1778
2    1038
Name: label, dtype: int64
